<a href="https://colab.research.google.com/github/sbc806/cpsc_533y_project/blob/master/run_pointnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/yanx27/Pointnet_Pointnet2_pytorch.git

Cloning into 'Pointnet_Pointnet2_pytorch'...
remote: Enumerating objects: 842, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 842 (delta 7), reused 11 (delta 5), pack-reused 826
Receiving objects: 100% (842/842), 68.77 MiB | 22.30 MiB/s, done.
Resolving deltas: 100% (485/485), done.


In [ ]:
!git clone https://github.com/sbc806/cpsc_533y_project.git

Cloning into 'cpsc_533y_project'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 34 (delta 11), reused 6 (delta 3), pack-reused 10
Receiving objects: 100% (34/34), 266.66 KiB | 6.20 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [ ]:
import sys as sys

sys.path.append("/content/cpsc_533y_project")

In [ ]:
import sys as sys

sys.path.append("/content/Pointnet_Pointnet2_pytorch")

In [ ]:
import sys as sys

sys.path.append("/content/Pointnet_Pointnet2_pytorch/models")

Read in data

In [ ]:
!pip install open3d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.5/420.5 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 103.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 92.6 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
  Attempting uninstall: nbformat
    Found existing installation: nbformat 5.9.2
    Uninstalling nbformat-5.9.2:
      Successfully uninstalled n

In [ ]:
!pip install tdqm

  Preparing metadata (setup.py) ... done
  Created wheel for tdqm: filename=tdqm-0.0.1-py3-none-any.whl size=1322 sha256=207d10c12c17b2adaa442b4a6d55552a81f4d6d5f59cfd2b22a1349debc3a9c5
  Stored in directory: /root/.cache/pip/wheels/37/31/b8/7b711038035720ba0df14376af06e5e76b9bd61759c861ad92
Successfully built tdqm


## Copy data from original directory to /cpsc_533y_project/data

In [ ]:
!cp -r /content/drive/MyDrive/cpsc_533y/project_datasets/modelnet40_ply_hdf5_2048 /content/cpsc_533y_project/data

# Get configuration

In [ ]:
!pip install ml_collections==0.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ml_collections: filename=ml_collections-0.1.1-py3-none-any.whl size=94505 sha256=97176221e7f65b190840d3ce0e96a8065054f2fe8c3867de5e26a278c90ead6a
  Stored in directory: /root/.cache/pip/wheels/7b/89/c9/a9b87790789e94aadcfc393c283e3ecd5ab916aed0a31be8fe
Successfully built ml_collections


In [ ]:
import ml_collections

In [ ]:
def get_config():
    config = ml_collections.ConfigDict()

    config.mode = "train"

    # Hyperparameters for dataset.
    config.num_pts = 64
    config.num_classes = 40
    config.use_voxel = False
    config.rot_type = "SO3"
    config.rot = False
    config.normalize = False
    config.center = False
    config.preprocess = True

    # Hyperparameters for models.

    # Hyperparameters for training.
    config.log_dir = "logs"
    config.use_cuda = True
    config.batch_size = 16
    config.num_epochs = 200
    config.lr = 0.02
    config.label_smoothing = 0.20

    # Hyperparameters for ablation study
    config.mode = "both"
    return config

In [ ]:
config = get_config()

In [ ]:
config

batch_size: 16
center: false
label_smoothing: 0.2
log_dir: logs
lr: 0.02
mode: both
normalize: false
num_classes: 40
num_epochs: 200
num_pts: 64
preprocess: true
rot: false
rot_type: SO3
use_cuda: true
use_voxel: false

# Get dataloaders

In [ ]:
def normalize_batch_data(batch_data):
    """ Normalize the batch data, use coordinates of the block centered at origin,
        Input:
            BxNxC array
        Output:
            BxNxC array
    """
    B, N, C = batch_data.shape
    normal_data = np.zeros((B, N, C))
    for b in range(B):
        pc = batch_data[b]
        centroid = np.mean(pc, axis=0)
        pc = pc - centroid
        m = np.max(np.sqrt(np.sum(pc ** 2, axis=1)))
        pc = pc / m
        normal_data[b] = pc
    return normal_data

In [ ]:
# From ScanObjectNN GitHub
def normalize_data(pcs):
    for pc in pcs:
        d = max(np.sum(np.abs(pc)**2, axis=-1)**(1./2))
        # print(d)
        pc /= d
    return pcs

In [ ]:
# From ScanObjectNN GitHub
def center_data(pcs):
    for pc in pcs:
        centroid = np.mean(pc, axis=0)
        pc[:0]-=centroid[0]
        pc[:,1]-=centroid[1]
        pc[:,2]-=centroid[2]
        # print(centroid)
    return pcs

In [ ]:
# Make anyadjustments to the hyperparameters for the dataloaders such as the number of points or rotation
config.num_pts = 16
config.batch_size = 24
config.rot = True
config.preprocess = False
config.batch_preprocess = True

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
import torch.nn.functional as F
# from torchvision import transforms, utils
from torch.utils.data import Subset

import time
import numpy as np
# from model import Reconstruction, TriangleNet
from dataloader import load_data, ModelNetDataLoader
from tqdm import tqdm
import argparse
# parser = argparse.ArgumentParser('Triangle-Net')
# parser.add_argument('--batch_size', type=int, default=32, help='batch size')
# parser.add_argument('--datapath', type=str, default=r'./data/modelnet40_ply_hdf5_2048/', help='path of modelnet 40 dataset')
# parser.add_argument('--episodes', type=int, default=1000)
# parser.add_argument('--n_points', type=int, default=16)
# parser.add_argument('--descriptor_type', type=str, default='C', help='[A, B, C]')
# parser.add_argument('--rot_type', type=str, default='SO3', help='[SO3, z]')
# args = parser.parse_args()

# datapath = args.datapath
# batch_size = args.batch_size
# train_episodes = args.episodes
# descriptor_type = args.descriptor_type
# n_points = args.n_points
# rot_type = args.rot_type

print(config)

# Adjust path to modelnet40_ply_hdf5_2048 directory as necessary
datapath = "/content/cpsc_533y_project/data/modelnet40_ply_hdf5_2048/"
n_points = config.num_pts
use_voxel = config.use_voxel
rot_type = config.rot_type
rot = config.rot

train_data, train_label, test_data, test_label = load_data(datapath, classification=True)
if config.center or config.preprocess:
    print("Centering training data")
    train_data = center_data(train_data)
    print("Centering test data")
    test_data = center_data(test_data)
if config.normalize or config.preprocess:
    print("Normalizing training data")
    train_data = normalize_data(train_data)
    print("Normalizing test data")
    test_data = normalize_data(test_data)

if config.batch_preprocess and not config.preprocess:
    print("Preprocessing batch training data")
    train_data = normalize_batch_data(train_data)
    print("Preprocessing batch test data")
    test_data = normalize_batch_data(test_data)

trainDataset = ModelNetDataLoader(train_data, train_label, use_voxel=use_voxel, point_num = n_points, rot_type=rot_type,rot=rot)
testDataset = ModelNetDataLoader(test_data, test_label, use_voxel=use_voxel, point_num = n_points, rot_type=rot_type,rot=rot)

weights = torch.ones(len(trainDataset))
random_indices = torch.multinomial(weights, len(trainDataset), replacement=False)
val_proportion = 0.20
num_val = int(len(trainDataset) * val_proportion)

trainDataLoader = torch.utils.data.DataLoader(trainDataset, batch_size=config.batch_size, shuffle=True) #, num_workers = 6
# valDataLoader = torch.utils.data.DataLoader(Subset(trainDataset, random_indices[0: num_val]), batch_size=config.batch_size, shuffle=False)
testDataLoader = torch.utils.data.DataLoader(testDataset, batch_size=config.batch_size, shuffle=False) #, num_workers = 6

# Create a single batch to overfit on
num_all=4
weights = torch.ones(len(trainDataset))
random_indices=torch.multinomial(weights,num_all,replacement=False)
batch_subset = Subset(trainDataset, random_indices)
batchTrainDataLoader=torch.utils.data.DataLoader(batch_subset,batch_size=4,shuffle=True)
batchTestDataLoader=torch.utils.data.DataLoader(batch_subset,batch_size=4,shuffle=False)

batch_preprocess: true
batch_size: 24
center: false
label_smoothing: 0.2
log_dir: logs
lr: 0.02
mode: both
normalize: false
num_classes: 40
num_epochs: 200
num_pts: 16
preprocess: false
rot: true
rot_type: SO3
use_cuda: true
use_voxel: false

Preprocessing batch training data
Preprocessing batch test data


In [ ]:
print(len(trainDataset[0]))

3


In [ ]:
data = next(iter(testDataLoader))
# First element is the point cloud, second element is the estimated normals, third element is the labels
# If use_voxel=True, third elment will be the voxels, and fourth element will be the labels
print(len(data))

3


In [ ]:
pc = data[0]
print(f"Point cloud batch shape: {pc.shape}")

Point cloud batch shape: torch.Size([24, 16, 3])


# Run PointNet

In [ ]:
"""
Author: Benny
Date: Nov 2019
"""

import os
import sys
import torch
import numpy as np

import datetime
import logging
import provider
import importlib
import shutil
import argparse

from pathlib import Path
from tqdm import tqdm
# from data_utils.ModelNetDataLoader import ModelNetDataLoader

# BASE_DIR = os.path.dirname(os.path.abspath(__file__))
# ROOT_DIR = BASE_DIR
# sys.path.append(os.path.join(ROOT_DIR, 'models'))

def parse_args():
    '''PARAMETERS'''
    parser = argparse.ArgumentParser('training')
    parser.add_argument('--use_cpu', action='store_true', default=False, help='use cpu mode')
    parser.add_argument('--gpu', type=str, default='0', help='specify gpu device')
    parser.add_argument('--batch_size', type=int, default=24, help='batch size in training')
    parser.add_argument('--model', default='pointnet_cls', help='model name [default: pointnet_cls]')
    parser.add_argument('--num_category', default=40, type=int, choices=[10, 40],  help='training on ModelNet10/40')
    parser.add_argument('--epoch', default=200, type=int, help='number of epoch in training')
    parser.add_argument('--learning_rate', default=0.001, type=float, help='learning rate in training')
    parser.add_argument('--num_point', type=int, default=1024, help='Point Number')
    parser.add_argument('--optimizer', type=str, default='Adam', help='optimizer for training')
    parser.add_argument('--log_dir', type=str, default=None, help='experiment root')
    parser.add_argument('--decay_rate', type=float, default=1e-4, help='decay rate')
    parser.add_argument('--use_normals', action='store_true', default=False, help='use normals')
    parser.add_argument('--process_data', action='store_true', default=False, help='save data offline')
    parser.add_argument('--use_uniform_sample', action='store_true', default=False, help='use uniform sampiling')
    return parser.parse_args()


def inplace_relu(m):
    classname = m.__class__.__name__
    if classname.find('ReLU') != -1:
        m.inplace=True


def test(model, loader, num_class=40, use_cpu=True):
    mean_correct = []
    class_acc = np.zeros((num_class, 3))
    classifier = model.eval()

    for j, (points, _, target) in tqdm(enumerate(loader), total=len(loader)):

        if not use_cpu:
            points, target = points.cuda(), target.cuda()

        points = points.transpose(2, 1)
        pred, _ = classifier(points)
        pred_choice = pred.data.max(1)[1]

        target = target.squeeze(-1)
        for cat in np.unique(target.cpu()):
            classacc = pred_choice[target == cat].eq(target[target == cat].long().data).cpu().sum()
            class_acc[cat, 0] += classacc.item() / float(points[target == cat].size()[0])
            class_acc[cat, 1] += 1

        correct = pred_choice.eq(target.long().data).cpu().sum()
        mean_correct.append(correct.item() / float(points.size()[0]))

    class_acc[:, 2] = class_acc[:, 0] / class_acc[:, 1]
    class_acc = np.mean(class_acc[:, 2])
    instance_acc = np.mean(mean_correct)

    return instance_acc, class_acc


def log_string(str):
    logger.info(str)
    print(str)

def main(args):
    def log_string(str):
        logger.info(str)
        print(str)

    '''HYPER PARAMETER'''
    os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu

    '''CREATE DIR'''
    timestr = str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M'))
    exp_dir = Path('./log/')
    exp_dir.mkdir(exist_ok=True)
    exp_dir = exp_dir.joinpath('classification')
    exp_dir.mkdir(exist_ok=True)
    if args.log_dir is None:
        exp_dir = exp_dir.joinpath(timestr)
    else:
        exp_dir = exp_dir.joinpath(args.log_dir)
    exp_dir.mkdir(exist_ok=True)
    checkpoints_dir = exp_dir.joinpath('checkpoints/')
    checkpoints_dir.mkdir(exist_ok=True)
    log_dir = exp_dir.joinpath('logs/')
    log_dir.mkdir(exist_ok=True)

    '''LOG'''
    args = parse_args()
    logger = logging.getLogger("Model")
    logger.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    file_handler = logging.FileHandler('%s/%s.txt' % (log_dir, args.model))
    file_handler.setLevel(logging.INFO)
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    log_string('PARAMETER ...')
    log_string(args)

    '''DATA LOADING'''
    log_string('Load dataset ...')
    data_path = 'data/modelnet40_normal_resampled/'

    train_dataset = ModelNetDataLoader(root=data_path, args=args, split='train', process_data=args.process_data)
    test_dataset = ModelNetDataLoader(root=data_path, args=args, split='test', process_data=args.process_data)
    trainDataLoader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=10, drop_last=True)
    testDataLoader = torch.utils.data.DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, num_workers=10)


In [ ]:
'''MODEL LOADING'''
num_class = 40
model = importlib.import_module("pointnet_cls")
# from models import pointnet_utils
# from models import pointnet_cls
# from models import pointnet_utils
classifier = model.get_model(num_class, normal_channel=False)
criterion = model.get_loss()
classifier.apply(inplace_relu)

use_cpu=False
if not use_cpu:
    classifier = classifier.cuda()
    criterion = criterion.cuda()

# try:
    # checkpoint = torch.load(str(exp_dir) + '/checkpoints/best_model.pth')
    # start_epoch = checkpoint['epoch']
    # classifier.load_state_dict(checkpoint['model_state_dict'])
    # log_string('Use pretrain model')
# except:
    # log_string('No existing model, starting training from scratch...')
    # start_epoch = 0

start_epoch = 0
optimizer = "Adam"
learning_rate = 0.001
decay_rate=1e-4
if optimizer == 'Adam':
    optimizer = torch.optim.Adam(
    classifier.parameters(),
    lr=learning_rate,
    betas=(0.9, 0.999),
    eps=1e-08,
            weight_decay=decay_rate
        )
else:
    optimizer = torch.optim.SGD(classifier.parameters(), lr=0.01, momentum=0.9)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.7)
global_epoch = 0
global_step = 0
best_instance_acc = 0.0
best_class_acc = 0.0

total_epochs = 200
'''TRANING'''
print('Start training...')
for epoch in range(start_epoch, total_epochs):
    print('Epoch %d (%d/%s):' % (global_epoch + 1, epoch + 1, epoch))
    mean_correct = []
    classifier = classifier.train()

    # scheduler.step()
    for batch_id, (points, _, target) in tqdm(enumerate(trainDataLoader, 0), total=len(trainDataLoader), smoothing=0.9):
        optimizer.zero_grad()

        points = points.data.numpy()
        points = provider.random_point_dropout(points)
        points[:, :, 0:3] = provider.random_scale_point_cloud(points[:, :, 0:3])
        points[:, :, 0:3] = provider.shift_point_cloud(points[:, :, 0:3])
        points = torch.Tensor(points)
        points = points.transpose(2, 1)

        if not use_cpu:
            points, target = points.cuda(), target.cuda()

        pred, trans_feat = classifier(points)
        loss = criterion(pred, target.long().squeeze(-1), trans_feat)
        pred_choice = pred.data.max(1)[1]

        correct = pred_choice.eq(target.long().squeeze(-1).data).cpu().sum()
        mean_correct.append(correct.item() / float(points.size()[0]))
        loss.backward()
        optimizer.step()
        global_step += 1

    scheduler.step()

    train_instance_acc = np.mean(mean_correct)
    print('Train Instance Accuracy: %f' % train_instance_acc)

    state = {
        'epoch': epoch,
        'model_state_dict': classifier.state_dict(),
        'optimizer_state_dict': optimizer,
        'config': config,
    }
    torch.save(state, "/content/drive/MyDrive/cpsc_533y/pointnet_16_rot_batch_center_normalize_checkpoint.pth")

    with torch.no_grad():
        instance_acc, class_acc = test(classifier.eval(), testDataLoader, num_class=num_class, use_cpu=use_cpu)

        if (instance_acc >= best_instance_acc):
            best_instance_acc = instance_acc
            best_epoch = epoch + 1

        if (class_acc >= best_class_acc):
            best_class_acc = class_acc
        print('Test Instance Accuracy: %f, Class Accuracy: %f' % (instance_acc, class_acc))
        print('Best Instance Accuracy: %f, Class Accuracy: %f' % (best_instance_acc, best_class_acc))

        if (instance_acc >= best_instance_acc):
            state = {
                'epoch': epoch,
                'model_state_dict': classifier.state_dict(),
                'optimizer_state_dict': optimizer,
                'config': config,
            }
            torch.save(state, "/content/drive/MyDrive/cpsc_533y/pointnet_16_rot_batch_center_normalize_best.pth")

        # if (instance_acc >= best_instance_acc):
            # logger.info('Save model...')
            # savepath = str(checkpoints_dir) + '/best_model.pth'
            # log_string('Saving at %s' % savepath)
            # state = {
                # 'epoch': best_epoch,
                # 'instance_acc': instance_acc,
                # 'class_acc': class_acc,
                # 'model_state_dict': classifier.state_dict(),
                # 'optimizer_state_dict': optimizer.state_dict(),
            # }
            # torch.save(state, savepath)
            # global_epoch += 1

print('End of training...')

print()
instance_acc, class_acc = test(classifier.eval(), testDataLoader, num_class=num_class, use_cpu=use_cpu)
print('Test Instance Accuracy: %f, Class Accuracy: %f' % (instance_acc, class_acc))

# main(args)

Start training...
Epoch 1 (1/0):


100%|██████████| 410/410 [01:16<00:00,  5.39it/s]


Train Instance Accuracy: 0.088720


100%|██████████| 103/103 [00:18<00:00,  5.43it/s]


Test Instance Accuracy: 0.051375, Class Accuracy: 0.032230
Best Instance Accuracy: 0.051375, Class Accuracy: 0.032230
Epoch 1 (2/1):


...

Train Instance Accuracy: 0.225508


100%|██████████| 103/103 [00:01<00:00, 56.99it/s]


Test Instance Accuracy: 0.272896, Class Accuracy: 0.225941
Best Instance Accuracy: 0.276861, Class Accuracy: 0.231818
End of training...



100%|██████████| 103/103 [00:02<00:00, 46.16it/s]

Test Instance Accuracy: 0.271359, Class Accuracy: 0.226514


# Load in pre-trained model

In [ ]:
import torch
import importlib

num_class = 40
model = importlib.import_module("pointnet_cls")

# Load pre-trained model
pre_trained_classifier = model.get_model(num_class, normal_channel=False)
criterion = model.get_loss()
pre_trained_classifier.apply(inplace_relu)

use_cpu = False
if not use_cpu:
    pre_trained_classifier = pre_trained_classifier.cuda()
    criterion = criterion.cuda()

checkpoint = torch.load("/content/drive/MyDrive/cpsc_533y/pointnet_1024_no_rot_normalize_checkpoint.pth")
pre_trained_classifier.load_state_dict(checkpoint["model_state_dict"])

<All keys matched successfully>

# Visualization functions

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio

In [ ]:
def interactive_3d_visualization(embeddings, desired_obs_all):

  # Create a meshgrid for the angles
  # phi, theta = np.linspace(0, np.pi, 100), np.linspace(0, 2 * np.pi, 100)
  # PHI, THETA = np.meshgrid(phi, theta)

  # Convert angles to Cartesian coordinates
  # x = np.sin(PHI) * np.cos(THETA)
  # y = np.sin(PHI) * np.sin(THETA)
  # z = np.cos(PHI)

  x = embeddings[:, 0]
  y = embeddings[:, 1]
  z = embeddings[:, 2]
  print(min(x))
  print(max(x))
  print(min(y), max(y))
  print(min(z), max(z))
  # Create the 3D scatter plot
  fig = go.Figure()
  fig.add_trace(go.Scatter3d(x=x,
                               y=y,
                               z=z,
                               mode='markers',
                               ))

  # Set plot title and axis labels
  fig.update_layout(
    # title='Interactive 3D Sphere',
      scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
      )
  )

  fig.show()
  # Save the figure as an interactive HTML file
  # pio.write_html(fig, file=os.path.join(figure_save_path, f'{dataset_name}_{model_name}_{desired_obs}.html'))

  # Save the figure as an image (e.g., PNG)
  # pio.write_image(fig, file='output_plot.png', format='png')


In [ ]:
interactive_3d_visualization(train_data[106],"test")

-0.83714527
0.8551462
-0.14703815 0.33583084
-0.9998928 0.9316149


In [ ]:
interactive_3d_visualization(batch_data[0][3],"test")

tensor(-0.9551)
tensor(0.6685)
tensor(-0.8961) tensor(0.6999)
tensor(-0.5538) tensor(0.5305)
